In [23]:
data = {
    "total": 4914,
    "start": "20050809",
    "name": "\u745e\u58f0\u79d1\u6280",
    "marketType": "",
    "sortYear": [[2005, 100], [2006, 247], [2007, 246], [2008, 247], [2009, 249], [2010, 249], [2011, 246], [2012, 247], [2013, 244], [2014, 247], [2015, 247], [2016, 245], [2017, 246], [2018, 246], [2019, 246], [2020, 248], [2021, 247], [2022, 244], [2023, 243], [2024, 246], [2025, 134]],
    "priceFactor": 1000
}

In [6]:
(len(data['sortYear']),len(data['price'])/len(data['afterVolumn']),len(data['volumn'])/len(data['afterVolumn']),len(data['afterVolumn'])/len(data['afterVolumn']),len(data['dates'])/len(data['afterVolumn']))

(21, 18.39242825157745, 8.047221656828821, 1.0, 5.000814166497048)

In [99]:
dates = []
date_parts = data["dates"].split(",")
current_index = 0

for year, count in data["sortYear"]:
    # 计算当前年份对应的日期片段范围
    start_idx = current_index
    end_idx = current_index + count
    
    # 提取对应的日期片段并与年份组合
    parts_for_year = date_parts[start_idx:end_idx]
    full_dates = [f"{year}{part}" for part in parts_for_year]
    
    # 更新索引并扩展结果列表
    current_index = end_idx
    dates.extend(full_dates)
reversed_dates = dates[::-1]

price_list = data['price'].split(',')
prices = [
    { 
        "open": (float(v[1]) + float(v[0]))/1000,
        "close": (float(v[3]) + float(v[0]))/1000,
        "high": (float(v[2]) + float(v[0]))/1000,
        "low": float(v[0])/1000
    } 
    for v in [tuple(price_list[i:i+4]) for i in range(0, len(price_list), 4)]
]
reversed_prices = prices[::-1]

volumn = data["volumn"].split(",")
reversed_volumn = volumn[::-1]

min_length = min(len(reversed_prices), len(reversed_dates), len(reversed_volumn))
merged_data = [
    {
        "date": reversed_dates[i],
        "volumn": reversed_volumn[i],
        **reversed_prices[i]  # 合并价格字典中的键值对
    }
    for i in range(0, min_length)
]

In [96]:
print(len(reversed_prices), len(reversed_dates), len(reversed_volumn))

4914 4914 4914


In [100]:
merged_data[0]

{'date': '20250721',
 'volumn': '7735170',
 'open': 40.95,
 'close': 39.9,
 'high': 41.35,
 'low': 39.6}

In [69]:
for x in range(0,5):
 print(x)

0
1
2
3
4


In [77]:
reversed_prices[0],reversed_dates[0],len(reversed_prices),len(reversed_dates),len(grouped_prices),len(dates)

({'open': -8.589, 'close': -8.289, 'high': -8.164, 'low': -8.589},
 '20050809',
 4914,
 4893,
 4914,
 4893)

In [47]:
range(cnt_min)

range(0, 4893)

In [22]:
# 39600,1350,1750,300
# 最低，今开（最低+该值）,最高（最低+该值），今收（最低+300）
l = 39600
o = l+ 1350
h
c = l + 300
# 7735170  成交量
39600 + 1750
# 40500,1250,1800,300,


41350

In [10]:
v = (39600,1350,1750,300)
l, o, h , c = v[0],v[0] + v[1], v[0] + v[2], v[0] + v[3]

In [11]:
l, o, h , c 

(39600, 40950, 41350, 39900)

In [12]:
o,c,h,l

(40950, 39900, 41350, 39600)